# Clipping plane visualization of Burgers tent slab solution

We show how to convert a tent slab based on a 2D mesh into a 3D mesh for which we can use the webgui's clipping feature.

In [1]:
from netgen.geom2d import unit_square
from ngsolve import Mesh, CoefficientFunction, x, y, exp, L2, H1, GridFunction, TaskManager
import ngsolve as ng
from ngstents import TentSlab
from ngstents.conslaw import Burgers
from ngsolve.webgui import Draw
from ngstents.utils import SlabConverter
import tentswebgui

In [2]:
def MakeTentSlab(mesh, dt):
    heapsize = 10*1000*1000
    c = 6
    ts = TentSlab(mesh, heapsize=heapsize)
    ts.SetMaxWavespeed(c)
    ts.PitchTents(dt)
    print("max slope", ts.MaxSlope())
    return ts

### Create and visualize (if desired) the tent slab

In [3]:
maxh = 0.05
mesh = Mesh(unit_square.GenerateMesh(maxh=maxh))
dt = 0.05
ts = MakeTentSlab(mesh, dt)
# tentswebgui.Draw(ts)

max slope 0.23812861119774045


### Construct and setup the Burgers instance

In [4]:
order = 4
V = L2(mesh, order=order)
u = GridFunction(V,"u")

burg = Burgers(u, ts, outflow=mesh.Boundaries("left|bottom|right|top"))
burg.SetTentSolver("SARK",substeps=order*order)

cf = CoefficientFunction(exp(-50*((x-0.3)*(x-0.3)+(y-0.3)*(y-0.3))))
burg.SetInitial(cf)


### Convert the tent slab into a periodic 3D mesh and define a space and GridFunction on it

In [5]:
order_3d = 2 # must be 1 or 2
sc = SlabConverter(ts, p_hd=order_3d)
sc.Convert(tscale=5)

# Define an H1 space on the 3D mesh and get its GridFunction.
V3d = H1(sc.mesh, order=order_3d)
u3d = GridFunction(V3d, "u3d")

# Set the data structure which maps 2D dofs for a front to 3D dofs
burg.SetIdx3d(sc.gfixmap)

# Store vectors for time steps which we want to examine the solution using clipping
# vecs = {1: u3d.vec.CreateVector(), 10: u3d.vec.CreateVector()}
vecs = {1: u3d.vec.CreateVector(), 2: u3d.vec.CreateVector(),
        3: u3d.vec.CreateVector(), 4: u3d.vec.CreateVector(),
        5: u3d.vec.CreateVector(), 6: u3d.vec.CreateVector(),
        7: u3d.vec.CreateVector(), 8: u3d.vec.CreateVector(),
        9: u3d.vec.CreateVector(), 10: u3d.vec.CreateVector()}

for v in vecs.values():
    v[:] = 0

add vertices 0.06399
add volume elements 0.17545
add surface elements 0.20745
Handle periodicity 0.00001
make ngsolve mesh 0.02071
make index map 0.34796977043151855
3621 verts, 17092 vol elems, 3998 surf elems in 0.81623.


### Run the Burgers simulation, saving vectors for timesteps of interest

In [6]:
tend = 10*dt
t = 0
n = 1

scene = Draw(u)
with TaskManager():
    while t < tend - dt/2:
        if n in vecs:
            # While propagating the solution, update the provided GridFunction vector.
            u3d.vec.data = vecs[n]
            burg.Propagate(hdgf=u3d)  # Propagate updates GridFunction u3d 
            vecs[n].data = u3d.vec    # Store updated solution vector for later viewing
        else:
            burg.Propagate()
        t += dt
        n += 1
        scene.Redraw()

WebGuiWidget(value={'ngsolve_version': '6.2.2103-39-gd1df1e2f3', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2, 'd…

## Now we can visualize the solution inside the slab by clipping the 3D mesh

In [7]:
u3d.vec.data = vecs[10]
clip = {'vec': [0, 0, -1], 'dist': 0.01} 
Draw(u3d, clipping=clip)

WebGuiWidget(value={'ngsolve_version': '6.2.2103-39-gd1df1e2f3', 'mesh_dim': 3, 'order2d': 2, 'order3d': 2, 'd…

BaseWebGuiScene